In [1]:
!pip install transformers

In [17]:
import nltk
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input
from google.colab import files
from tensorflow.keras.layers import Reshape




In [4]:
uploaded=files.upload()

Saving edos_labelled_aggregated.csv to edos_labelled_aggregated (1).csv


In [5]:
# Load your dataset and preprocess it (assuming you have a DataFrame 'df' with 'text' and 'label' columns)
data = pd.read_csv('edos_labelled_aggregated.csv',nrows=1000)

In [7]:
# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# Text cleaning and tokenization
def clean_text(text):
    text = text.lower()
    text = ' '.join(word for word in text.split() if word.isalnum())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

data['text'] = [clean_text(text) for text in data['text']]

In [8]:
# Converting labels to binary values
label_encoder = LabelEncoder()
data['label_exist'] = label_encoder.fit_transform(data['label_sexist'])

In [9]:
# Split your data into train, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data['text'], data['label_exist'], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)


In [27]:
# Load a pre-trained BERT model and tokenizer
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)

# Tokenize and pad the text data for input to the BERT model
max_length = 128  # You can adjust this based on your dataset
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
print(train_encodings )
print(val_encodings)
print(test_encodings)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

{'input_ids': <tf.Tensor: shape=(640, 128), dtype=int32, numpy=
array([[ 101, 2699, 2632, ...,    0,    0,    0],
       [ 101, 2123, 2102, ...,    0,    0,    0],
       [ 101, 5310, 8042, ...,    0,    0,    0],
       ...,
       [ 101, 3198, 3622, ...,    0,    0,    0],
       [ 101, 3035, 2155, ...,    0,    0,    0],
       [ 101, 6638, 2308, ...,    0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(640, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(640, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=

In [26]:
# Create a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).shuffle(100).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(64)
print(train_dataset)
print(val_dataset)
print(test_dataset)

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [12]:
# Define the BERT input
bert_input = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
bert_output = bert_model(bert_input)["pooler_output"]

# Reshape the BERT output to meet Conv1D requirements
bert_output = bert_model(dict(train_encodings))
reshape_layer = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x['last_hidden_state'], axis=-1))(bert_output)



In [21]:
# Repeat the second tensor to match the shape of the first tensor
tensor2_repeated = RepeatVector(128)(bert_output[1])

# Now, both tensors should have the same shape along axis 1
# Concatenate the two tensors
concatenated_tensor = Concatenate()([bert_output[0], tensor2_repeated])

# Additional layers for your CNN
conv1d = Conv1D(128, 5, activation='relu')(concatenated_tensor)
global_max_pooling = GlobalMaxPooling1D()(conv1d)
dense = Dense(64, activation='relu')(global_max_pooling)
output = Dense(1, activation='sigmoid')(dense)



In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, RepeatVector, Concatenate, Conv1D, GlobalMaxPooling1D, Dense
# Create the model
# Wrap the output tensor in a Keras layer
output_layer = tf.keras.layers.Lambda(lambda x: x)(output)
model = Model(inputs=bert_input, outputs=output)



ValueError: ignored

In [ ]:
# Compile the model with appropriate loss and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model with appropriate batch size, and other parameters
model.fit(train_dataset, validation_data=val_dataset, epochs=5, batch_size=32)  # Adjust batch size as needed



In [ ]:
# Evaluate the model
y_pred = model.predict(test_dataset)
y_pred = (y_pred > 0.5).astype(int)

# Assuming you have test_labels for evaluation
f1 = f1_score(test_labels, y_pred)
print("F1 Score:", f1)

In [20]:
# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=5)




NameError: ignored

In [ ]:
# Evaluate the model
y_pred = model.predict(test_dataset)
y_pred = (y_pred > 0.5).astype(int)
f1 = f1_score(test_labels, y_pred)
print("F1 Score:", f1)